# Synopsis
The model must be able to read in a sequence of numbers, one number at a time, and classify that number based on all previously seen numbers.

## Project TODO Steps
 1. create dataset
  - create single signal generator class
  - create mixed signal generator class
  - create noise functions (Gaussian, OU, etc.)
 2. create network
 3. train network
 4. test network

## Parameters to test
 1. sequence length
 2. amplitude
 3. frequency (period)
 4. offset
 5. phase
 6. noise
 
## Questions to answer
 1. What would a batch look like?
 2. How many LSTM layers do we need? nodes per LSTM layer?
 3. Can we learn if our timestamps are not uniformly spaced.

In [ ]:
import numpy as np
import pandas as pd
from random import random

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Masking
from keras.utils.np_utils import to_categorical

In [ ]:
SMALL_SIZE = 10
MEDIUM_SIZE = 12
BIGGER_SIZE = 16
BIGGEST_SIZE = 20
plt.rc('font', size=BIGGEST_SIZE)         # controls default text sizes
plt.rc('axes', titlesize=BIGGEST_SIZE)    # fontsize of the axes title
plt.rc('axes', labelsize=BIGGEST_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=BIGGER_SIZE)   # fontsize of the tick labels
plt.rc('ytick', labelsize=BIGGER_SIZE)   # fontsize of the tick labels
plt.rc('legend', fontsize=MEDIUM_SIZE)   # legend fontsize
plt.rc('figure', titlesize=BIGGEST_SIZE)  # fontsize of the figure title

## Build a signal generator

In [ ]:
from discrimnn.signal import MixedSignal

In [ ]:
# start off with simplest case for proof of concept
n_timesteps = 301

sig1_coeffs = {'phase': {'delta': np.pi}, 'amplitude': {'mean': 1.0}, 'period': {'mean': 50.0}, 'offset': {'mean': 0.5}}
sig2_coeffs = {'phase': {'delta': np.pi}, 'amplitude': {'mean': 1.0}, 'period': {'mean': 50.0}, 'offset': {'mean': 0.0}}
sig3_coeffs = {'phase': {'delta': np.pi}, 'amplitude': {'mean': 1.0}, 'period': {'mean': 50.0}, 'offset': {'mean': -0.5}}
sig_coeffs = [sig1_coeffs, sig2_coeffs, sig3_coeffs]
msig_coeffs = {'phase': {'mean': 0, 'delta': np.pi}}
msig = MixedSignal(0, 150, n_timesteps, sig_coeffs, msig_coeffs)
n_signals = len(msig)
X, y = msig()

print(X.shape, y.shape)

for i in range(n_signals):
    plt.plot(msig.timestamps, msig.signals[i], marker='.')

In [ ]:
def highlight_column_matches(data, column='', color='yellow'):
    '''
    highlight the maximum in a Series or DataFrame
    '''
    attr = 'background-color: {}'.format(color)
    if data.ndim == 1:  # Series from .apply(axis=0) or axis=1
        is_mixed = data == data[column]
        return [attr if v else '' for v in is_mixed]
    else:  # from .apply(axis=None)
        is_mixed = data == data[column]
        return pd.DataFrame(np.where(is_mixed, attr, ''), index=data.index, columns=data.columns)
    
df = pd.DataFrame(np.vstack((msig.signals, msig.mixed_signal)).T, index=msig.timestamps, columns=['A', 'B', 'C', 'Mixed'])
df[:5].style.apply(highlight_column_matches, column='Mixed', color='lightblue', axis=1)

In [ ]:
plt.plot(msig.timestamps, msig.mixed_signal, color='grey', alpha=0.3)
plt.scatter(msig.timestamps, msig.mixed_signal, marker='.', c=msig.labels)

In [ ]:
# define model
model = Sequential()
# model.add(Masking(mask_value=-1, input_shape=(1,)))
# model.add(LSTM(5))
# model.add(LSTM(5, input_shape=(2, 1)))
model.add(LSTM(32, return_sequences=True, input_shape=(1, 1)))
model.add(LSTM(64, return_sequences=True))
model.add(LSTM(32))
# model.add(LSTM(5, input_shape=(n_timesteps, 2)))
model.add(Dense(n_signals, activation='softmax'))


# model.add(LSTM(32, return_sequences=True, input_shape=(timesteps, data_dim)))
# model.add(LSTM(32, return_sequences=True))
# model.add(LSTM(32))

# model.add(LSTM(2048, return_sequences=False, input_shape=self.input_shape, dropout=0.5))
# model.add(Dense(512, activation='relu'))
# model.add(Dropout(0.5))

model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
# model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

model.summary()
for i in range(10):
#     X, y = msig.generate_batch(64)
    X, y = msig.generate()
    model.fit(X, y, epochs=20, batch_size=1, verbose=1, shuffle=False)
    model.reset_states()
#     model.fit(X, y, epochs=1, batch_size=1, verbose=2)

x_test, y_test = msig.generate()
score = model.evaluate(x_test, y_test, batch_size=1)
print(score)

In [ ]:
y_hat = model.predict(x_test)
for i in range(len(x_test)):
    print('Expected', y_test[i], 'Predicted', y_hat[i])

In [ ]:
x_test, y_test = msig.generate()
print(x_test.shape)
y_hat = model.predict(x_test)
print(y_hat.shape)
y_pred = np.argmax(y_hat, axis=1)
print(y_pred.shape)
y_error = np.max(y_hat, axis=1)
# y_error = msig.labels - y_hat
print(y_error.shape)
plt.plot(msig.timestamps, x_test[:, 0, 0], color='grey', alpha=0.3)
plt.scatter(msig.timestamps, x_test[:, 0, 0], marker='.', c=y_pred)
plt.show()

plt.plot(msig.timestamps, y_error, color='grey', alpha=0.3)
plt.scatter(msig.timestamps, y_error, marker='.', c=y_error)
plt.show()